In [1]:
import numpy as np
import pandas as pd

In [2]:
f = open('new_user_artists_scaled.txt', 'r')

In [3]:
arr = []
for s in f:
    arr.append(s.split('\t'))

In [4]:
weights = []
for i in range(len(arr)):
    arr[i][0] = int(arr[i][0])
    arr[i][1] = int(arr[i][1])
    arr[i][2] = float(arr[i][2])
    weights.append(arr[i][2])

In [5]:
df = pd.DataFrame(arr, columns=['uid', 'iid', 'rate'])

In [6]:
df

,uid,iid,rate
0,2,51,3.733741
1,2,52,3.666446
2,2,53,3.654928
3,2,54,3.616898
4,2,55,3.563351
5,2,56,3.415189
6,2,57,3.402451
7,2,58,3.302770
8,2,59,3.278371
9,2,60,3.260839


In [7]:
rmin = min(df["rate"])
rmax = max(df["rate"])
rmid = (rmin + rmax) / 2.0

In [8]:
print(rmin)
print(rmax)
print(rmid)

0.271325
5.0
2.6356625


In [9]:
def Agreement(r1, r2):
    if ((r1 > rmid) and (r2 < rmid)) or ((r1 < rmid) and (r2 > rmid)):
        return False
    return True

In [10]:
def Proximity(r1,r2):
    if Agreement(r1,r2):
        absDist = abs(r1-r2)
    absDist = 2*abs(r1-r2)
    return ((2 * (rmax - rmin) + 1) - absDist)**2

In [11]:
def Impact(r1,r2):
     return (((abs(r1 - rmid) + 1)*(abs(r2 - rmid) + 1)) 
             if Agreement(r1,r2) 
             else (1.0 / ((abs(r1 - rmid) + 1) * (abs(r2 - rmid) + 1))))

In [12]:
def Popularity(arr, r1, r2, item):
    itemRating = []
    for i in range(len(arr)):
        if arr[i][1] == item:
            itemRating.append(arr[i][2])
                
    avg = np.mean(itemRating)
    return ((1 + (((r1 + r2)/2.0 - avg) ** 2)) 
            if (r1 > avg and r2 > avg) or (r1 < avg and r2 < avg) 
            else 1)

In [13]:
def PIP(arr, df, user1, user2):
    commonItem = {}
    
    u1_items = df.loc[df["uid"] == user1, "iid"]
    u2_items = df.loc[df["uid"] == user2, "iid"]
    for item in u1_items:
        if item in u2_items:
            commonItem[item] = 1
                     
    if len(commonItem)==0: return 0
       
    result = 0
    for item in u1_items:
        if item in u2_items:
            r1 = df.loc[(df["uid"] == user1)&(df["iid"] == item), "rate"].sum()
            r2 = df.loc[(df["uid"] == user2)&(df["iid"] == item), "rate"].sum()
            result += Proximity(r1, r2) * Impact(r1, r2) * Popularity(arr, r1, r2, item)

    return result

In [16]:
users = []
users = df["uid"]
users = set(users)
users = list(users)

In [18]:
len(users)

1885

In [19]:
PIP(arr, df, 2, 4)

2.811455
0.0


5.467365435604077

In [ ]:
PIP_mat = np.zeros((len(users), len(users)))

for i in range(0, len(users)):
    for j in range(i, len(users)):
        PIP_mat[i][j] = PIP(arr, df, users[i], users[j])
        PIP_mat[j][i] = PIP_mat[i][j]

3.733741
0.0
3.666446
0.0
3.654928
0.0
3.616898
0.0
3.563351
0.0
3.415189
0.0
3.402451
0.0
3.30277
0.0
3.278371
0.0
3.260839
0.0
3.239109
0.0
3.224784
0.0
3.219891
0.0
3.210238
0.0
3.203195
0.0
3.172855
0.0
3.171553
0.0
3.124498
0.0
3.095798
0.0
3.090876
0.0
3.086186
0.0
3.080992
0.0
3.075727
0.0
3.070084
0.0
3.046334
0.0
3.043878
0.0
2.998286
0.0
2.998101
0.0
2.973527
0.0
2.969972
0.0
2.965181
0.0
2.948775
0.0
2.932525
0.0
2.921005
0.0
2.897372
0.0
2.88672
0.0
2.878035
0.0
2.876526
0.0
2.867867
0.0
2.855306
0.0
2.846431
0.0
2.839017
0.0
2.837906
0.0
2.828338
0.0
2.825479
0.0
2.819405
0.0
2.817945
0.0
2.81648
0.0
2.815892
0.0
2.811455
0.0
3.713283
0.0
2.543094
0.0
2.427917
0.0
2.375421
0.0
2.34919
0.0
2.298578
0.0
2.191443
0.0
2.140414
0.0
2.104096
0.0
2.104096
0.0
2.098621
0.0
2.062047
0.0
2.04975
0.0
2.005721
0.0
1.996288
0.0
1.974195
0.0
1.966545
0.0
1.942653
0.0
1.942653
0.0
1.893111
0.0
1.847006
0.0
1.836378
0.0
1.821743
0.0
1.802645
0.0
1.798711
0.0
1.782567
0.0
1.761403
0.0
1.76

In [ ]:
PIP_mat